# Flask 만들기
- 서비스 구동

## 라이브러리 설치

In [1]:
# 플라스크
!pip install flask
!pip install flask_cors
!pip install requests

# 구글 드라이브
!pip install --upgrade google-api-python-client

# 셀레니움
!pip install selenium

# DB
!pip install pymysql



  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/4d/b6/dccec9357261452eeaeb1900245b1100cff3b25c2005149432cc1b71a1c2/google_api_python_client-2.110.0-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ------ --------------------------------- 2.1/12.9 MB 43.8 MB/s eta 0:00:01
   ------------------ --------------------- 6.0/12.9 MB 63.6 MB/s eta 0:00:01
   -------------------------------- ------- 10.4/12.9 MB 93.9 MB/s eta 0:00:01
   ---------------------------------------  12.9/12.9 MB 93.9 MB/s eta 0:00:01
   ---------------------------------------  12.9/12.9 MB 93.9 MB/s eta 0:00:01
   ---------------------------------------- 12.9/12.9 MB 50.3 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.109.0
    Uninstalling google-api-python-client-2.109.0:
      Successfully uninstalled google-api-python-clie

In [2]:
# 라이브러리 임폴트
from flask import Flask, request, make_response, redirect
from flask_cors import CORS

# 영상처리를 위한 라이브러리
import cv2
import mediapipe as mp
import numpy as np 
import matplotlib.pyplot as plt
import math
import imutils
from roboflow import Roboflow

from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

## 필요한 함수 정의

### 이메일 전송

In [3]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def sendEmail(to_mail, mem_id, mem_pw):
    my_mail = "gkfzkseka@naver.com"
    pwd = "chlwlsrbs1!"
    to_mail = "gkfzkseka@naver.com"  

    msg = MIMEMultipart()
    msg['Subject'] = "네일뀽"  
    msg['From'] = my_mail
    msg['To'] = to_mail

    text = MIMEText("귀하의 아이디는 "+mem_id+"이고 비밀번호는 "+mem_pw+"입니다.")
    msg.attach(text)

    smtp = smtplib.SMTP("smtp.naver.com", 587)
    smtp.starttls()
    smtp.login(user=my_mail, password=pwd)
    smtp.sendmail(my_mail, to_mail, msg.as_string())
    smtp.close()


### 네일아트 서비스

In [4]:
def nailArt(hand_url, nail_url):
    # 사진 읽어오기
    img = cv2.imread(hand_url)               # 원본 사진을 img에 저장
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img_nail = cv2.imread(nail_url)               
    img_nail = cv2.cvtColor(img_nail, cv2.COLOR_BGR2RGB)

    img_height, img_width = img.shape[:2]               # 원본 사진의 높이와 너비를 저장
    centerX, centerY = img_width//2, img_height//2     # 원본 사진의 중앙값을 저장

    img_nail_height, img_nail_width = img_nail.shape[:2]               
    nail_centerX, nail_centerY = img_nail_width//2, img_nail_height//2     

    rf = Roboflow(api_key="0PeUxiGgkgDe4u1ykT9b")
    project = rf.workspace().project("nail-8e9wp")
    model = project.version(2).model

    result = model.predict(hand_url).json()
    sorted_result = sorted(result['predictions'], key=lambda x:(x['x'], x['y']))

    # mediapipe 탐색
    mpHands = mp.solutions.hands
    hands = mpHands.Hands()
    mpDraw = mp.solutions.drawing_utils

    results = hands.process(img)
    hand_list = []
    \
    img2 = img.copy()
    img_nail2 = img_nail.copy()
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for id, lm in enumerate(handLms.landmark):
                cx, cy = int(lm.x *img_width), int(lm.y *img_height)
                hand_list.append([cx, cy])
                cv2.circle(img2, (cx, cy), 10, (0, 0, 255), cv2.FILLED)

            mpDraw.draw_landmarks(img2, handLms, mpHands.HAND_CONNECTIONS)

    # 엄지 : 3, 4
    # 검지 : 7, 8
    # 중지 : 11, 12
    # 약지 : 15, 16
    # 소지 : 19, 20

    nailpoint = [[3, 4], [7, 8], [11, 12], [15, 16], [19, 20]]

    # 손가락에 대해서 각각 rotate 및 범위 지정
    # for p in nailpoint:
    img_hand = img.copy()

    for i in range(5):
        p = nailpoint[4-i]
        robo_centerX = sorted_result[i]['x']
        robo_centerY = sorted_result[i]['y']

        pA_x, pA_y = hand_list[p[0]]
        pB_x, pB_y = hand_list[p[1]]

        rad = math.atan2(pA_x - pB_x, pA_y - pB_y)
        deg = (rad*180)/3.14


        min_x = 100000
        min_y = 100000
        max_x = -100000
        max_y = -100000
        for p in sorted_result[0]['points']:

            nx = round(math.cos(rad)*p['x'] - math.sin(rad)*p['y'])
            ny = round(math.sin(rad)*p['x'] + math.cos(rad)*p['y'])

            min_x = min(min_x, nx)
            min_y = min(min_y, ny)
            max_x = max(max_x, nx)
            max_y = max(max_y, ny)

        resize_x = max_x-min_x
        nail_height, nail_width = img_nail.shape[:2]
        resize_y = int(nail_height * (resize_x/nail_width))

        img_nail_copy = img_nail.copy()
        img_nail_resized = cv2.resize(img_nail_copy, (resize_x, resize_y))


        M = cv2.getRotationMatrix2D((centerX, centerX), -deg, 1.0)
        rimg = cv2.warpAffine(img2, M, (img_width, img_height))

        rimg_nail = imutils.rotate_bound(img_nail_resized, -deg)
        rimg_nail_height, rimg_nail_width = rimg_nail.shape[:2]



        for y in range(rimg_nail_height):
            for x in range(rimg_nail_width):
                if rimg_nail[y][x][0] < 10 and rimg_nail[y][x][1] < 10 and rimg_nail[y][x][2] < 10:
                    rimg_nail[y][x][0] = 255
                    rimg_nail[y][x][1] = 255
                    rimg_nail[y][x][2] = 255

        rA_x, rA_y = int((pA_x-centerX)*math.cos(math.radians(-rad)) - (pA_y-centerY)*math.sin(math.radians(-rad))), int((pA_x-centerX*math.sin(math.radians(-rad)) + (pA_y-centerY)*math.cos(math.radians(-rad))))
        rA_x, rA_y = rA_x + centerX, rA_y + centerY

        rB_x, rB_y = int((pB_x-centerX)*math.cos(math.radians(-rad)) - (pB_y-centerY)*math.sin(math.radians(-rad))), int((pB_x-centerX*math.sin(math.radians(-rad)) + (pB_y-centerY)*math.cos(math.radians(-rad))))
        rB_x, rB_y = rB_x + centerX, rB_y + centerY

        mask_nail = rimg_nail

        # sub_img의 크기는 mask_nail의 크기만큼
        mask_height, mask_width = mask_nail.shape[:2]
        startX = int(robo_centerX - (mask_width//2))
        startY = int(robo_centerY - (mask_height//2))

        try:
            mask2gray = cv2.cvtColor(mask_nail, cv2.COLOR_BGR2GRAY)
            _, mask_b = cv2.threshold(mask2gray, 250, 255, cv2.THRESH_BINARY)
            mask_b_inv = cv2.bitwise_not(mask_b)


            sub_img = img_hand[ startY:startY+mask_height , startX:startX+ mask_width]
            img_bg = cv2.bitwise_and(sub_img,sub_img,mask = mask_b)
            img_fg = cv2.bitwise_and(mask_nail,mask_nail, mask=mask_b_inv)
            bg_fg = cv2.add(img_bg,img_fg)
            img_hand[ startY:startY+mask_height , startX:startX+ mask_width] = bg_fg

            plt.subplot(1, 5, i+1) 
            plt.xticks([]), plt.yticks([])
            plt.title("Nail")
            plt.imshow(img_hand)
        except:
            print("예외발생")

    img_result = cv2.cvtColor(img_hand, cv2.COLOR_RGB2BGR)
    cv2.imwrite("img/result.jpg", img_result)    
    
    return img_result



In [5]:
# 현재 파일 개수 반환
def len_file():
    driver = wb.Chrome() 

    url = 'https://drive.google.com/drive/folders/1QK2S0KR3pN-3iMF6Da4ONUkJHZAzSq0U/'

    driver.get(url)
    # xpath = '//*[@id=":1"]/div/c-wiz/div[2]/c-wiz/div[1]/c-wiz/div[2]/c-wiz/div[1]/c-wiz/c-wiz/div/'
    classname = 'pmHCK'
    elements = driver.find_elements(By.CLASS_NAME, classname)
    print(len(elements))
    
    driver.close()
    
    return len(elements)



In [6]:
len_file()

The chromedriver version (119.0.6045.105) detected in PATH at C:\Users\smhrd\NailProject\chromedriver.exe might not be compatible with the detected chrome version (120.0.6099.71); currently, chromedriver 120.0.6099.71 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry


46


46

In [7]:
# file_id 가져오기
def select_fileId(fname):
    driver = wb.Chrome() 

    if fname == 'NailartImg':
        url = 'https://drive.google.com/drive/folders/1QK2S0KR3pN-3iMF6Da4ONUkJHZAzSq0U/'
    elif fname == 'ShopImg':
        url = 'https://drive.google.com/drive/folders/1t5x1Sj6BcZToO9TN9qLHdkFsBrCnmFVq/'

    driver.get(url)
    # xpath = '//*[@id=":1"]/div/c-wiz/div[2]/c-wiz/div[1]/c-wiz/div[2]/c-wiz/div[1]/c-wiz/c-wiz/div/'
    classname = 'pmHCK'
    elements = driver.find_elements(By.CLASS_NAME, classname)
    index = (len(elements))
    print(index)
    
    xpath = '//*[@id=":1"]/div/c-wiz/div[2]/c-wiz/div[1]/c-wiz/div[2]/c-wiz/div[1]/c-wiz/c-wiz/div/c-wiz['+str(index)+']/div'
    element = driver.find_element(By.XPATH, xpath)
  
    file_id = element.get_attribute('data-id')
   
    driver.close()
    
    return file_id

In [8]:
select_fileId("ShopImg")

The chromedriver version (119.0.6045.105) detected in PATH at C:\Users\smhrd\NailProject\chromedriver.exe might not be compatible with the detected chrome version (120.0.6099.71); currently, chromedriver 120.0.6099.71 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry


11


'1nuw1wQ5nqKpZNPgBYM4pfYldf1FZRMg5'

In [9]:
# file_id 가져오기
def select_fileId2(fname):
    driver = wb.Chrome() 

    if fname == 'NailartImg':
        url = 'https://drive.google.com/drive/folders/1QK2S0KR3pN-3iMF6Da4ONUkJHZAzSq0U/'
    elif fname == 'ShopImg':
        url = 'https://drive.google.com/drive/folders/1t5x1Sj6BcZToO9TN9qLHdkFsBrCnmFVq/'

    driver.get(url)
    # xpath = '//*[@id=":1"]/div/c-wiz/div[2]/c-wiz/div[1]/c-wiz/div[2]/c-wiz/div[1]/c-wiz/c-wiz/div/'
    classname = 'pmHCK'
    elements = driver.find_elements(By.CLASS_NAME, classname)
    index = (len(elements))
    print(index)
    
    for i in range(1, index+1):
        xpath = '//*[@id=":1"]/div/c-wiz/div[2]/c-wiz/div[1]/c-wiz/div[2]/c-wiz/div[1]/c-wiz/c-wiz/div/c-wiz['+str(i)+']/div'
        element = driver.find_element(By.XPATH, xpath)

        file_id = element.get_attribute('data-id')
        print(file_id)
   
    driver.close()
    
    return file_id

In [10]:
select_fileId2("ShopImg")

The chromedriver version (119.0.6045.105) detected in PATH at C:\Users\smhrd\NailProject\chromedriver.exe might not be compatible with the detected chrome version (120.0.6099.71); currently, chromedriver 120.0.6099.71 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry


11
1t0UC74vERR86wza4sR_xr9imE3bJjQvU
1kfGIPtB_f_RExPDL9n7qOE7_PCnnLeG2
1HJnap8Bz9dKj7-xGp-NLjE5MpVKosu7Y
1hz9DcnVIWWfUvRRP1mFKflOLwicqq7ff
1YxShNU5dqaN1pUkmmocxY3ur5Ir3l7c7
1IRLqcnSqqatb1SeDVrk3qyXT4E0sCs_N
1Ywnnn-_n5S29sy5Z8o5q-xZlUSrAFlEz
1LfJCLmWLVz_FOvyXj9ZdLUWvX1ulh0U5
1hZbnhTLk8pJ4GdbJt7bK8KJzElQJbSPK
18vmGcun_pR0ZKPMrjjjqvPwtovZQoroL
1nuw1wQ5nqKpZNPgBYM4pfYldf1FZRMg5


'1nuw1wQ5nqKpZNPgBYM4pfYldf1FZRMg5'

### 파일 다운로드

In [11]:
#import gdown
#driver = wb.Chrome() 

#url = 'https://drive.google.com/drive/folders/1QK2S0KR3pN-3iMF6Da4ONUkJHZAzSq0U/'

#driver.get(url)

#for i in range(1, 43, 1):
#    xpath = '//*[@id=":1"]/div/c-wiz/div[2]/c-wiz/div[1]/c-wiz/div[2]/c-wiz/div[1]/c-wiz/c-wiz/div/c-wiz['+str(i)+']/div'
#    element = driver.find_element(By.XPATH, xpath)
#    print(element.get_attribute('data-id'))


In [12]:
# 

In [13]:
# NailartImg 폴더 내부의 공유파일 다운로드
def file_download(fname, index):
    import gdown
    driver = wb.Chrome() 
    url = ''
    
    if fname == 'NailartImg':
        url = 'https://drive.google.com/drive/folders/1QK2S0KR3pN-3iMF6Da4ONUkJHZAzSq0U/'
        
    driver.get(url)
    xpath = '//*[@id=":1"]/div/c-wiz/div[2]/c-wiz/div[1]/c-wiz/div[2]/c-wiz/div[1]/c-wiz/c-wiz/div/c-wiz['+index+']/div'
    element = driver.find_element(By.XPATH, xpath)
    print(element.get_attribute('data-id'))
    
    file_id = element.get_attribute('data-id')
    print(file_id)
    
    output = './img/download.png' # 저장 위치 및 저장할 파일 이름
    gdown.download(id=file_id, output=output, quiet=False)
    
    driver.close()
    
    return output

In [14]:
# test
print(file_download("NailartImg", "1"))

The chromedriver version (119.0.6045.105) detected in PATH at C:\Users\smhrd\NailProject\chromedriver.exe might not be compatible with the detected chrome version (120.0.6099.71); currently, chromedriver 120.0.6099.71 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry


11lcbK9sQvQExIe2fuoWte6UfboQYtJrV
11lcbK9sQvQExIe2fuoWte6UfboQYtJrV


Downloading...
From: https://drive.google.com/uc?id=11lcbK9sQvQExIe2fuoWte6UfboQYtJrV
To: C:\Users\smhrd\NailProject\img\download.png
100%|█████████████████████████████████████████████████████████████████████████████| 29.4k/29.4k [00:00<00:00, 1.54MB/s]


./img/download.png


In [15]:
# NailartImg 폴더 내부의 공유파일 다운로드
def file_download2(fild_id):
    import gdown
    output = './img/download.png' # 저장 위치 및 저장할 파일 이름
    gdown.download(id=fild_id, output=output, quiet=False)

    return output

In [16]:
# test
# print(file_downlboad2('1oixA42pnhd7qDVWGm9GSADnX1wKBIDUB'))

In [17]:
!pip install argparse

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


### 파일 업로드

In [18]:
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools
import argparse

def file_upload(fname, iname):
    flags = argparse.ArgumentParser(parents=[tools.argparser]).parse_args(args=[])

    SCOPES = 'https://www.googleapis.com/auth/drive.file'
    store = file.Storage('storage.json')
    creds = store.get()

    if not creds or creds.invalid:
        print("make new storage data file ")
        flow = client.flow_from_clientsecrets('client_secret_drive.json', SCOPES)
        creds = tools.run_flow(flow, store, flags) \
                if flags else tools.run(flow, store)

    DRIVE = build('drive', 'v3', http=creds.authorize(Http()))

    FILES = (
        (iname),
    )

    if fname == "NailartImg":
        folder_id = '1QK2S0KR3pN-3iMF6Da4ONUkJHZAzSq0U'
    elif fname == "ShopImg":
        folder_id = '1t5x1Sj6BcZToO9TN9qLHdkFsBrCnmFVq' 

    for file_title in FILES :
        file_name = file_title
        metadata = {'name': file_name[6:],
                    'parents' : [folder_id],
                    'mimeType': None
                    }

        res = DRIVE.files().create(body=metadata, media_body=file_name).execute()
        if res:
            print('Uploaded "%s" (%s)' % (file_name, res['mimeType']))

In [19]:
file_upload("NailartImg", "./img/nail1.png")

Uploaded "./img/nail1.png" (image/png)


In [20]:
!pip install jsonify

## Flask 서버 구동

In [ ]:
from flask import Flask, request, make_response, redirect
from flask_cors import CORS
from flask import Flask, render_template, send_file
import cv2
from flask import Blueprint, request
from werkzeug.utils import secure_filename
from flask import Flask, jsonify

bp = Blueprint('image', __name__, url_prefix='/image')

app = Flask(__name__)

CORS(app)

# 서버연결 테스트용으로 만든 메시지를 그대로 되돌려주는 함수
@app.route("/", methods=["GET", "POST"]) 
def startFlask():
    msg = request.args.get("message") # --> request.getParameter("name값")
    return dict({"res":msg})

# 전송한 파일을 로컬에 저장하는 함수
@app.route('/saveInputfile', methods=['POST'])
def saveInputfile():
    if request.method =='POST':
        f = request.files['fileInput']
        save_to = f'img/input_file.png'
        f.save(save_to)
        return 'success'
    
# 전송한 파일을 로컬에 저장하는 함수
@app.route('/saveInputfile2', methods=['POST'])
def saveInputfile2():
    if request.method =='POST':
        f = request.files['input_file']
        save_to = f'img/input_file2.png'
        f.save(save_to)
        return 'success'
    
# 전송한 파일을 로컬에 저장하는 함수
# 위의 함수랑 같은건데 흠....
# 
@app.route('/saveNailImg', methods=['POST'])
def saveNailImg():
    if request.method =='POST':
        f = request.files['input_file']
        name = request.files['name']
        save_to = f'img/{name}'
        f.save(save_to)
        return 'success'

# 입력값. 사용자의 손 사진을 받아서 저장 
@app.route('/file', methods=['POST'])
def file_post():
    input_img = request.form.get('input_file')
    PATH = "img/input_img.jpg"
    #cv2.imwrite(PATH, input_img)
    
    #img = nailArt(PATH, './img/nail1.png')
    
    return request.form.get('title')

# 테스트 하는 코드
# 뭘 하려했던 건지 알 수 없음
@app.route("/test", methods=["GET", "POST"]) 
def test():
    if request.method == 'POST':
        result = request.form
        return render_template("8_sending_form_data_result.html", result=result)

# 사용자의 손 사진을 읽고
# 사용자가 요청한 네일아트 사진을 읽어와서 네일아트 서비스를 실행
@app.route("/nailArtService", methods=["GET", "POST"]) 
def nailArtService():
    # hand_img = request.args.get("hand_img")
    nail_img = request.args.get("nail_img")
    PATH = file_download2(nail_img)
    
    img = nailArt("./img/input_file.png", PATH)

    PATH = "./img/nail_result.png"
    cv2.imwrite(PATH, img)

    return send_file(PATH, as_attachment=True)
    
@app.route("/nailArtService2", methods=["GET", "POST"]) 
def nailArtService2():
    # hand_img = request.args.get("hand_img")
    nail_img = request.args.get("nail_img")
    nail_img = "./img/"+nail_img[7:]
    print(nail_img)

    img = nailArt("./img/input_file.png", nail_img)

    PATH = "./img/nail_result.png"
    cv2.imwrite(PATH, img)

    return send_file(PATH, as_attachment=True)

@app.route("/GoogleDriveUpload", methods=["GET", "POST"]) 
def GoogleDriveUpload():
    if request.method =='POST':
        f = request.files['input_file']
        fname = request.form['fname']
        iname = request.form['iname']
        
        save_to = f'img/{iname}'
        f.save(save_to)
        
        PATH = f'./img/{iname}'
        # 저장 시킨 다음에 
        file_upload(fname, PATH)
        
        file_id = select_fileId(fname)
    
    return file_id
    
@app.route("/GoogleDriveDownload", methods=["GET", "POST"]) 
def GoogleDriveDownload():
    img_idx = request.args.get("img_idx")
    PATH = file_download('NailartImg', img_idx)
    
    return send_file(PATH, as_attachment=True)

@app.route("/GoogleDriveDownload2", methods=["GET", "POST"]) 
def GoogleDriveDownload2():
    nail_img = request.args.get("nail_img")
    PATH = file_download2(nail_img)
    
    res = send_file(PATH, as_attachment=True)
    return res
    #return nail_img

@app.route("/GoogleDriveDownload3", methods=["GET", "POST"]) 
def GoogleDriveDownload3():
    nail_id = request.args.get("nail_id")
    nail_img = request.args.get("nail_img")
    PATH = file_download2(nail_img)
    
    img = cv2.imread(PATH)
    
    #return jsonify({'nail_id' : nail_id, 'nail_img': img})
    res = send_file(PATH, as_attachment=True)
    return jsonify({'nail_id' : nail_id, 'nail_img' : res})
    
# to_email에게 mem_id와 mem_pw를 전송
@app.route("/sendEmailService", methods=["GET", "POST"]) 
def sendEmailService():
    to_email = request.args.get("to_email")
    mem_id = request.args.get("mem_id")
    mem_pw = request.args.get("mem_pw")
    sendEmail(to_email, mem_id, mem_pw)
    return "success"

if __name__ == "__main__" : # main method 역할! --> 서버를 구동시키는 부분
    app.run(host = "127.0.0.1", port = "9003")


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:9003
Press CTRL+C to quit
Downloading...
From: https://drive.google.com/uc?id=1t0UC74vERR86wza4sR_xr9imE3bJjQvU
To: C:\Users\smhrd\NailProject\img\download.png
100%|███████████████████████████████████████████████████████████████████████████████| 694k/694k [00:00<00:00, 3.22MB/s]
127.0.0.1 - - [08/Dec/2023 10:38:25] "GET /GoogleDriveDownload2?nail_img=1t0UC74vERR86wza4sR_xr9imE3bJjQvU HTTP/1.1" 200 -
Downloading...
From: https://drive.google.com/uc?id=1t0UC74vERR86wza4sR_xr9imE3bJjQvU
To: C:\Users\smhrd\NailProject\img\download.png
100%|███████████████████████████████████████████████████████████████████████████████| 694k/694k [00:00<00:00, 3.00MB/s]
127.0.0.1 - - [08/Dec/2023 10:39:33] "GET /GoogleDriveDownload2?nail_img=1t0UC74vERR86wza4sR_xr9imE3bJjQvU HTTP/1.1" 200 -
Downloading...
From: https://drive.google.com/uc?id=1t0UC74vERR86wza4sR_xr9imE3bJjQvU
To: C:\Users\smhrd\NailProject\img\download.png
100%|██████████████████████████████████████████████████

loading Roboflow workspace...
loading Roboflow project...


127.0.0.1 - - [08/Dec/2023 10:44:58] "GET /nailArtService?nail_img=1BUhDDhtDM6rYsic1EQWzMVD6DOTZKFAp HTTP/1.1" 200 -
Downloading...
From: https://drive.google.com/uc?id=1t0UC74vERR86wza4sR_xr9imE3bJjQvU
To: C:\Users\smhrd\NailProject\img\download.png
100%|███████████████████████████████████████████████████████████████████████████████| 694k/694k [00:00<00:00, 3.15MB/s]
127.0.0.1 - - [08/Dec/2023 10:45:33] "GET /GoogleDriveDownload2?nail_img=1t0UC74vERR86wza4sR_xr9imE3bJjQvU HTTP/1.1" 200 -
Downloading...
From: https://drive.google.com/uc?id=1t0UC74vERR86wza4sR_xr9imE3bJjQvU
To: C:\Users\smhrd\NailProject\img\download.png
100%|███████████████████████████████████████████████████████████████████████████████| 694k/694k [00:00<00:00, 3.79MB/s]
127.0.0.1 - - [08/Dec/2023 10:46:01] "GET /GoogleDriveDownload2?nail_img=1t0UC74vERR86wza4sR_xr9imE3bJjQvU HTTP/1.1" 200 -
Downloading...
From: https://drive.google.com/uc?id=1t0UC74vERR86wza4sR_xr9imE3bJjQvU
To: C:\Users\smhrd\NailProject\img\download

loading Roboflow workspace...
loading Roboflow project...


...

127.0.0.1 - - [08/Dec/2023 16:00:23] "POST /GoogleDriveUpload HTTP/1.1" 200 -


Uploaded "./img/21성민샵.png" (image/jpeg)


The chromedriver version (119.0.6045.105) detected in PATH at C:\Users\smhrd\NailProject\chromedriver.exe might not be compatible with the detected chrome version (120.0.6099.71); currently, chromedriver 120.0.6099.71 is recommended for chrome 120.*, so it is advised to delete the driver in PATH and retry


13


127.0.0.1 - - [08/Dec/2023 16:01:15] "POST /GoogleDriveUpload HTTP/1.1" 200 -


# 함수 테스트

In [ ]:
# nailArt("./img/img2.jpg", "./img/nail1.png")

In [ ]:
# sendEmail("gkfzkseka@naver.com", "jkchoi", "1234")